In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_casa import casa
from casa import Cadet, Cadence
import re
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
import torch

In [3]:
cadet = Cadet.load(casa.get_data_path()/"cadet/op20")

[INFO] 2021-04-26 17:25:48,563 gensim.utils: loading KeyedVectors object from E:\LangOn\casa\data\cadet\op20\ft-2020.kv
[INFO] 2021-04-26 17:25:48,900 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-04-26 17:25:48,901 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-04-26 17:25:48,901 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-04-26 17:25:48,902 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-04-26 17:25:49,012 gensim.utils: FastTextKeyedVectors lifecycle event {'fname': 'E:\\LangOn\\casa\\data\\cadet\\op20\\ft-2020.kv', 'datetime': '2021-04-26T17:25:49.012564', 'gensim': '4.0.0', 'python': '3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.18362-SP0', 'event': 'loaded'}
[WARNING] 2021-04-26 17:25:49,019 gensim.models.keyedvectors: destructive init_sims(replace=True) deprecated & no longer required for space-eff

In [4]:
ckpt_path = casa.get_data_path() / "cadence/bert-sentiment-50.02.1/model.ckpt"
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels = 3)
model.load_state_dict(torch.load(ckpt_path))
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [5]:
import pandas as pd
cx_path = casa.get_data_path() / "constructicon/sentiment-constructicon.csv"
cx_df = pd.read_csv(cx_path, index_col=None).dropna()
cx_list = cx_df.to_dict(orient='list')
cx_list = [(cx, score) for cx, score in zip(*cx_list.values())]

## Cadence: **C**ht **A**bsa **DE**teciton with **N**eural network and **C**onstructicon **E**nsemble


## Samples

In [6]:
cadence = Cadence.build_Q2(cadet, model, tokenizer, cx_list)

In [7]:
cadence.analyze("中華在山上的訊號還不錯", summary=True)

'中華電信(0.94)/通訊品質(0.49)/Positive(0.91, Bert)'

In [8]:
cadence.analyze("只有遠傳沒有訊號", summary=True)

'遠傳電信(0.49)/通訊品質(0.96)/Negative(1.00, CxG)'

In [9]:
cadence.analyze("遠傳測速是在測心酸的", summary=False)

{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '其他電信'],
 'entity_probs': array([0.01198956, 0.01227514, 0.94205485, 0.01115532, 0.01108392,
        0.01144121]),
 'service': ['資費方案', '通訊品質', '加值服務', '其他'],
 'service_probs': array([7.82737379e-05, 9.99760666e-01, 8.33699990e-05, 7.76904188e-05]),
 'seeds': ['測試', 'Speed', '測速', '上傳', '下載'],
 'seed_probs': array([0.95608653, 0.00644206, 0.00644206, 0.00644206, 0.00644206]),
 'sentiment': ['Neutral', 'Positive', 'Negative'],
 'sentiment_src': 'CxG',
 'sentiment_probs': array([0., 0., 1.], dtype=float32)}

In [13]:
cadence.analyze("中華撿到槍", summary=True)

'中華電信(0.94)/資費方案(0.44)/Positive(0.51, Bert)'